In [ ]:
import pandas as pd
from datetime import datetime
pd.options.plotting.backend = "plotly"

zelty = pd.read_csv("../zelty/zelty.csv", sep=";",decimal=',',low_memory=False, index_col=False)
zelty['date'] = pd.to_datetime(zelty['date'])
zelty.head()

In [ ]:
chiffredaffaire = zelty.groupby(['date', 'category'])[['price']].sum().unstack('category')

In [ ]:
chiffredaffaire_par_an = chiffredaffaire.resample('YS').sum().fillna(0)
chiffredaffaire_par_an['year'] = chiffredaffaire_par_an.index.year
chiffredaffaire_par_an.reset_index(inplace=True)
chiffredaffaire_par_an.set_index('year', inplace=True)
# = chiffredaffaire.columns.droplevel(0)
chiffredaffaire_par_an.columns = chiffredaffaire_par_an.columns.droplevel(0)
del chiffredaffaire_par_an['']
chiffredaffaire_par_an.plot(kind="bar")

In [ ]:
chiffredaffaire_12_mois = chiffredaffaire.resample('MS').sum()
chiffredaffaire_12_mois = chiffredaffaire_12_mois[chiffredaffaire_12_mois.index > str(datetime.now().replace(year=datetime.now().year - 1))]
chiffredaffaire_12_mois.columns = chiffredaffaire_12_mois.columns.droplevel(0)
chiffredaffaire_12_mois.plot(kind='bar')

In [ ]:
chiffredaffaire_total_by_year = chiffredaffaire.stack('category').reset_index().groupby('date')[['price']].sum().resample('Y').sum()
chiffredaffaire_total_by_year.index = chiffredaffaire_total_by_year.index.year

chiffredaffaire_a_date = chiffredaffaire.stack('category').reset_index().groupby('date')[['price']].sum()
chiffredaffaire_a_date['a_date'] = chiffredaffaire_a_date.index.month*100+chiffredaffaire_a_date.index.day
chiffredaffaire_a_date = chiffredaffaire_a_date[chiffredaffaire_a_date['a_date'] < datetime.now().month*100+datetime.now().day]
del chiffredaffaire_a_date['a_date']
chiffredaffaire_a_date = chiffredaffaire_a_date.resample('Y').sum()
chiffredaffaire_a_date.index = chiffredaffaire_a_date.index.year
chiffredaffaire_a_date
libelle_a_date = "à date du %02d/%02d" % (datetime.now().day, datetime.now().month)
chiffredaffaire_total_by_year[libelle_a_date] = chiffredaffaire_a_date['price']
chiffredaffaire_total_by_year.fillna(0, inplace=True)
chiffredaffaire_total_by_year['toute l\'année'] = chiffredaffaire_total_by_year['price'] - chiffredaffaire_total_by_year[libelle_a_date]
del chiffredaffaire_total_by_year['price']
chiffredaffaire_total_by_year.plot(kind="bar")


In [ ]:
limit_date = datetime.now().replace(year=datetime.now().year - 1)
chiffredaffaire_top_year = chiffredaffaire[chiffredaffaire.index.to_period("M") >= str(limit_date.year)+'-'+str(limit_date.month)]
chiffredaffaire_top_year.columns = chiffredaffaire_top_year.columns.droplevel(0)
chiffredaffaire_top_year = chiffredaffaire_top_year.groupby('date').sum()
chiffredaffaire_top_year['total'] = pd.DataFrame(chiffredaffaire_top_year.stack('category'), columns=['total']).groupby('date').sum()
chiffredaffaire_top_year = chiffredaffaire_top_year.sort_values(by='total', ascending=False)
chiffredaffaire_top_year[['total']].head(20)


In [ ]:
del chiffredaffaire_top_year['total']
chiffredaffaire_top_year.index = chiffredaffaire_top_year.index.strftime('%d/%m/%Y')
chiffredaffaire_top_year.head(50).plot(kind='bar')

In [ ]:
benefices = chiffredaffaire.stack('category')[['price']].groupby('date').sum().resample('MS').sum()
benefices['year'] = benefices.index.year
marge = {2017:0.5,2018:0.5,2019:0.5,2020:0.35,2021:0.35,2022:0.35,2023:0.45}
benefices['marge'] = benefices['year'].map(marge)
benefices['charges_fixes'] = 4000
benefices = benefices.groupby(['date','year']).sum()
benefices['total'] = benefices['price'] * benefices['marge'] - benefices['charges_fixes']
del benefices['price']
del benefices['marge']
del benefices['charges_fixes']
benefices = benefices.unstack('year').fillna(0)
benefices.columns = benefices.columns.droplevel(0)
benefices.plot(kind='bar')

In [ ]:
benefices.sum()